In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp data

# Data

> Utilities for retrieving benchmarks and generating random graphs

In [3]:
#export 
from nbdev.showdoc import *
import numpy as np
import networkx as nx
import scipy
import pygsp
from pathlib import Path

## Benchmarks

These datasets are downloaded and preprocessed using https://github.com/shchur/gnn-benchmark. I aim to replace them once I work out how to make gnn-benchmark a dependency

In [4]:
#export
def get_benchmark(dataset):
    """Returns an adjacency matrix `A`, feature matrix `X` and labels `y` from standard benchmark.
    
    The data is normalised as done in `https://github.com/shchur/gnn-benchmark`.
    
    Args:
        `dataset` (string): Can be cora, pubmed, citeseer, amazon_electronics_photo, 
                            amazon_electronics_computers, ms_academic_phy or ms_academic_cs
                            
    Returns:
        `A`: An adjacency matrix in sparse array format  
        `X`: A feature matrix in sparse array format  
        `y`: Labels in list format
    """
    fname = Path(__file__).parents[1].joinpath(f'data/standardised_{dataset}.npz')
    data = np.load(fname, allow_pickle=True)
    A, X, y = data['A'].tolist(), data['X'].tolist(), data['y']
    return A, X, y

## Synethetic

In [5]:
#export

def get_planar_graph(n, return_pos=False):
    """Returns a networkx graph which is planar on n nodes.
    
    The generation process is taken from:
    https://stackoverflow.com/questions/26681899/how-to-make-networkx-graph-from-delaunay-preserving-attributes-of-the-input-node
    
    If `return_pos` a planar embedding is also returned in the form of {node : (x, y)}
    """
    points = np.random.rand(n, 2)
    delTri = scipy.spatial.Delaunay(points)
    edges = set()
    for n in range(delTri.nsimplex):
        edge = sorted([delTri.vertices[n,0], delTri.vertices[n,1]])
        edges.add((edge[0], edge[1]))
        edge = sorted([delTri.vertices[n,0], delTri.vertices[n,2]])
        edges.add((edge[0], edge[1]))
        edge = sorted([delTri.vertices[n,1], delTri.vertices[n,2]])
        edges.add((edge[0], edge[1]))
    graph = nx.Graph(list(edges))
    
    if return_pos:
        pos = dict(zip(range(len(points)), points))
        return graph, pos
    else:
        return graph
    
def get_sensor_graph(n):
    """Returns a networkx graph for a connected KNN sensor graph
    
    This used the github pygsp.graphs.Sensor implementation, not the stable release (i.e. as described in the docs).
    """
    G = pygsp.graphs.Sensor(n)
    while not G.is_connected():
        G = pygsp.graphs.Sensor(n)
    return nx.Graph(G.W)

def get_BASBM(sizes, p, m=2):
    """A stochastic block model where each block is a Barabasi Albert graph.
    
    Args:
        `sizes` (list): a list of ints describing the size of each block. Each 
            size must be larger than m+1.
        `p`: (float or array): if an array then p[i][j] is the probability of connecting
            a node from block i to block j. If a float then its the probability of connecting
            nodes from different blocks. 
        `m`: (int): the barabasi albert hyper-parameter.
        
    Returns:
        A networkx graph.
    """
    if isinstance(p, float) or isinstance(p, int):
        p = p*np.ones((len(sizes), len(sizes)))
    num_blocks = len(sizes)
    blocks = []
    for i in range(num_blocks):
        block_row = []
        for j in range(num_blocks):
            if i == j:
                block_row.append(nx.adj_matrix(nx.barabasi_albert_graph(sizes[i], m)).todense())
            else:
                block_row.append(np.random.binomial(1, p[i][j], (sizes[i], sizes[j])))
        blocks.append(block_row)
    adj_matrix = np.block(blocks)
    return nx.from_numpy_array(adj_matrix)

In [6]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_graphtools.ipynb.
Converted 01_sampling.ipynb.
Converted 02_metrics.ipynb.
Converted 03_perturb.ipynb.
Converted 04_plotting.ipynb.
Converted 05_data.ipynb.
Converted index.ipynb.


In [10]:
m = 2
sizes = [5, 5]
G = get_BASBM(sizes, 1, m)
expected_total_edges = np.sum([(n-m)*m for n in sizes])
for i in range(len(sizes)):
    for j in range(len(sizes)):
        if i!=j:
            print(sizes[i]*sizes[j])
            expected_total_edges += sizes[i]*sizes[j]
assert len(G.edges()) == expected_total_edges
    

25
25


AssertionError: 

In [9]:
nx.adj_matrix(G).todense()

matrix([[0, 0, 1, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
        [0, 1, 1, 0, 0, 1, 1, 1, 1, 1],
        [0, 1, 1, 0, 0, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 1, 1, 0],
        [1, 1, 1, 1, 1, 0, 0, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
        [1, 1, 1, 1, 1, 0, 0, 1, 1, 0]], dtype=int64)